<a href="https://colab.research.google.com/github/vlad98rus/-netology_pyda/blob/HomeWork-1/%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание «Коллаборативная фильтрация»


-  ПАКЕТ SURPRISE

-  используйте данные MovieLens 1M
-  можно использовать любые модели из пакета
-  получите RMSE на тестовом сете 0.87 и ниже
-  В ДЗ на датасет 1М можешь не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете

In [2]:
pip install SURPRISE

     |████████████████████████████████| 6.5MB 5.2MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675371 sha256=9fbedfab539b8f21954dfd110ebc4d3aa4b29553e716b4576ae2394c70b8dbb8
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, SVD, SVDpp
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import KFold

In [0]:
N_FOLDS = 5
RANDOM_STATE = 777

In [0]:
kfold = KFold(5)

In [0]:
df_ratings = pd.read_csv('ratings.csv', usecols=['uid', 'iid', 'rating'], names=['uid', 'iid', 'rating', 'to_drop'], header=1)

In [9]:
print(df_ratings.describe())

                 uid            iid         rating
count  100835.000000  100835.000000  100835.000000
mean      326.130788   19435.488451       3.501552
std       182.616527   35531.110673       1.042533
min         1.000000       1.000000       0.500000
25%       177.000000    1199.000000       3.000000
50%       325.000000    2991.000000       3.500000
75%       477.000000    8123.000000       4.000000
max       610.000000  193609.000000       5.000000


In [8]:
df_ratings.head()

,uid,iid,rating
0,1,3,4.0
1,1,6,4.0
2,1,47,5.0
3,1,50,5.0
4,1,70,3.0


In [0]:
reader = Reader(rating_scale=(0.5, 5))

In [0]:
dataset = Dataset.load_from_df(df_ratings, reader)

### Baseline с отложенной выборкой

In [0]:
trainset, testset = train_test_split(dataset, test_size=0.2)

In [14]:
algo = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)
predictions = algo.test(testset)
print(f'RMSE on TEST 20% = {accuracy.rmse(predictions)}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8799
RMSE on TEST 20% = 0.8798929898838946


### CV using 5 folds

In [15]:
scores = []
for trainset, testset in tqdm_notebook(kfold.split(dataset)):
    algo = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
    algo.fit(trainset)
    predictions = algo.test(testset)
    scores.append(accuracy.rmse(predictions))
print(f'Mean RMSE = {np.mean(scores)}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8888
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8835
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8791
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8813
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8749

Mean RMSE = 0.881532523041028


### Test other algos

In [16]:
algo = SVD()
cross_validate(algo, dataset, measures=['RMSE'], cv=N_FOLDS, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8748  0.8741  0.8707  0.8752  0.8708  0.8731  0.0020  
Fit time          4.98    4.99    4.97    4.99    5.04    4.99    0.02    
Test time         0.14    0.26    0.14    0.24    0.14    0.18    0.05    


{'fit_time': (4.981456995010376,
  4.989786863327026,
  4.970468997955322,
  4.985289812088013,
  5.037747859954834),
 'test_rmse': array([0.87477109, 0.87412221, 0.87067955, 0.87518416, 0.87081673]),
 'test_time': (0.14204192161560059,
  0.26220226287841797,
  0.13762879371643066,
  0.23896145820617676,
  0.14012932777404785)}

In [18]:
algo = SVDpp()
cross_validate(algo, dataset, measures=['RMSE'], cv=N_FOLDS, verbose=True)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8601  0.8572  0.8569  0.8581  0.8714  0.8607  0.0055  
Fit time          505.44  499.85  502.17  495.11  501.60  500.84  3.39    
Test time         10.20   9.97    10.15   9.84    10.32   10.10   0.17    


{'fit_time': (505.44087982177734,
  499.8538737297058,
  502.17478823661804,
  495.1089446544647,
  501.6017096042633),
 'test_rmse': array([0.86014862, 0.85717315, 0.85688387, 0.85805784, 0.87139633]),
 'test_time': (10.201176166534424,
  9.974907636642456,
  10.151615858078003,
  9.840617895126343,
  10.318268060684204)}

In [0]:
param_grid = {'n_factors': [50, 100, 200], 'n_epochs': [50, 100, 200], 'lr_all': [0.002, 0.005],
              'biased': [True, False], 'reg_all': [0.02, 0.1, 0.2, 0.4], 'random_state': [RANDOM_STATE], 'verbose': [True]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=N_FOLDS, n_jobs=-1, joblib_verbose=True)
gs.fit(dataset)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 20.5min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 71.3min


0.8488435089125215
{'n_factors': 200, 'n_epochs': 200, 'lr_all': 0.005, 'biased': True, 'reg_all': 0.1, 'random_state': 777, 'verbose': True}


[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 177.6min finished
